In [6]:
import pandas as pd
from datetime import datetime, timezone, timedelta
# obtain the all_regional_mtlf_after_intg.parquet file from 
# https://www.kaggle.com/dataset/fa253a48766d3621ee0cd56fa478ed464af7672d22b10cf7e9f294da728063b0
file_name = 'all_regional_mtlf_after_intg.parquet'
df = pd.read_parquet(f'../data/{file_name}')
target_range = df[(df['Market Day'] >= datetime(2015,1,20)) & (df['Market Day'] <= datetime(2021,12,31))]

In [7]:

targets = pd.DataFrame()

targets['MarketHour'] = target_range.apply(lambda x: x['Market Day'].replace( hour = x['HourEnding'] - 1, tzinfo = timezone(timedelta(hours=-5))), axis=1)
targets['Load'] = target_range['MISO ActualLoad (MWh)']
targets.set_index('MarketHour', inplace=True)

In [8]:
weather = pd.read_parquet('../data/weather/WI_Y51_weather_data.parquet')
weather.set_index('valid', inplace=True)

In [9]:
from datetime import timezone, timedelta

start = datetime(2015,  1, 20,  0, tzinfo=timezone(timedelta(hours=-5)))
stop  = datetime(2021, 12, 31, 23, tzinfo=timezone(timedelta(hours=-5)))
dates = pd.date_range(start = start, end = stop)
hours = [d.replace(hour = h) for d in dates for h in range(0, 23)]
hourly_weather = [weather.iloc[weather.index.get_loc(d, method='nearest')] for d in hours]

In [10]:
df = targets.merge(weather, left_index=True, right_index=True)

In [11]:
from numpy import linalg
linalg.lstsq(df[['lat', 'lon', 'feel']], df['Load'])

/home/ca/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  


(array([ 1.20594423e+17,  5.78177665e+16, -7.20000000e+01]),
 array([6.34258992e+09]),
 3,
 array([7.16499156e+02, 1.42711601e+02, 1.41979713e-13]))